In [1]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19878 sha256=04cc34985c0ed0f1b2e35671193d04b770fbf7f580018709ce48ef754e3fdaa6
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [2]:
import pandas as pd
from vecstack import stacking
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xg
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.neural_network import MLPClassifier
from collections import Counter
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount("/content/drive")
#Change current working directory to gdrive




/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be import

Mounted at /content/drive


In [3]:
#Loading the data
trainfile = r'/content/drive/MyDrive/CIS508/Assignment_3/RevisedHomesiteTrain1.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/content/drive/MyDrive/CIS508/Assignment_3/RevisedHomesiteTest1.csv'
testData = pd.read_csv(testfile)  #creates a dataframe

print(trainData.shape)
print(testData.shape)


(65000, 596)
(173836, 596)


**EDA**

In [4]:
#EDA_Train_Set
print(trainData.info())
print(trainData.isnull().sum().sum())
print(trainData.head())
trainData.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65000 entries, 0 to 64999
Columns: 596 entries, CoverageField11A to QuoteConversion_Flag
dtypes: float64(5), int64(591)
memory usage: 295.6 MB
None
0
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
count,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.0,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.00000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,...,65000.000000,65000.000000,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000
mean,7.798892,12.954892,9.014585,12.995831,9.060000,13.017308,9.064538,13.017877,8.973785,13.011031,13.814523,21.201046,11.465477,19.179246,953.073415,1.162747,18.262569,0.938339,0.001456,-1.0,25.0,7.370092,13.191985,10.711554,13.328754,7.781154,13.22280,-0.325600,13.964031,11.680662,13.316185,7.282015,13.181169,4.994108,12.984062,-0.333200,13.038754,11.867246,13.055738,7.318415,...,0.263846,0.063538,0.0,0.138262,0.111631,0.749954,0.000154,0.034877,0.934138,0.005231,0.025754,0.942938,0.057062,0.234092,0.038523,0.127277,0.600108,0.263923,0.735892,0.181385,0.036185,0.307969,0.474462,0.457569,0.542246,0.951569,0.048031,0.0,0.712754,0.287246,0.974969,0.020292,0.0,0.976262,0.023738,0.361862,0.120908,0.267631,0.249600,0.188646
std,5.304856,7.259023,5.586114,7.217093,5.568791,7.207118,5.588694,7.212825,5.564675,7.203325,5.105662,4.911451,6.893381,7.395318,290.710820,0.116634,7.405558,0.037123,0.001489,0.0,0.0,7.052165,7.157675,7.676760,7.201487,7.056926,7.11729,4.132777,6.076901,7.317099,7.222147,5.983242,7.121157,5.951460,7.191695,4.110041,7.141211,8.021028,7.219648,5.239252,...,0.440720,0.243931,0.0,0.345177,0.314914,0.433043,0.012403,0.183469,0.248042,0.072135,0.158401,0.231962,0.231962,0.423433,0.192457,0.333285,0.489880,0.440761,0.440860,0.385339,0.186751,0.461657,0.499351,0.498200,0.498216,0.214676,0.213833,0.0,0.452481,0.452481,0.156220,0.140999,0.0,0.152234,0.152234,0.480543,0.326022,0.442727,0.432785,0.391230
min,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,548.0

In [5]:
#EDA_Test_Set
print(testData.info())
print(testData.isnull().sum().sum())
print(testData.head())
testData.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173836 entries, 0 to 173835
Columns: 596 entries, CoverageField11A to GeographicField64
dtypes: float64(5), int64(590), object(1)
memory usage: 790.5+ MB
None
0
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_Q,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX
count,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.0,173836.00000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,...,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.000000,173836.0,173836.0,173836.0,173836.0
mean,7.793184,12.966060,9.018132,13.014893,9.063244,13.036494,9.061690,13.025231,8.974223,13.030074,13.799443,21.187430,11.455924,19.153294,955.236482,1.162914,18.288335,0.938323,0.001447,-1.0,24.99985,7.372995,13.175044,10.698618,13.310712,7.781466,13.196254,-0.318427,13.979596,11.647587,13.278544,7.289796,13.172358,5.032605,13.014617,-0.337721,13.062461,11.833751,13.018903,7.339740,...,0.023465,0.263904,0.066005,0.000006,0.137912,0.110903,0.751064,0.000121,0.035188,0.933213,0.004683,0.026916,0.944195,0.055800,0.234940,0.038364,0.128213,0.598484,0.262966,0.736798,0.182080,0.036517,0.308900,0.472503,0.460267,0.539497,0.952731,0.046883,0.000006,0.711930,0.288065,0.973164,0.021969,0.000006,0.977099,0.022895,0.0,0.0,0.0,0.0
std,5.277834,7.235874,5.563241,7.204874,5.544948,7.195815,5.568419,7.194741,5.540401,7.195347,5.106860,4.931883,6.917724,7.412874,290.925921,0.116912,7.385814,0.037059,0.001484,0.0,0.06236,7.053064,7.176345,7.687929,7.231301,7.059159,7.151810,4.154090,6.096448,7.332813,7.245175,5.999285,7.159907,5.968310,7.216944,4.096431,7.147985,8.013367,7.213383,5.269366,...,0.151374,0.440749,0.248291,0.002398,0.344808,0.314013,0.432398,0.010990,0.184256,0.249654,0.068269,0.161839,0.229546,0.229535,0.423962,0.192073,0.334328,0.490206,0.440246,0.440372,0.385911,0.187574,0.462041,0.499245,0.498420,0.498439,0.212214,0.211389,0.002398,0.452865,0.452863,0.161603,0.146583,0.002398,0.149588,0.149570,0.0,0.0,0.0,0.0
min,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1

In [ ]:
#Check the Column name
trainCols = list(trainData.columns.values)
testCols = list(testData.columns.values)
print(trainCols)
print(testCols)

['CoverageField11A', 'CoverageField11B', 'CoverageField1A', 'CoverageField1B', 'CoverageField2A', 'CoverageField2B', 'CoverageField3A', 'CoverageField3B', 'CoverageField4A', 'CoverageField4B', 'CoverageField5A', 'CoverageField5B', 'CoverageField6A', 'CoverageField6B', 'Field10', 'Field11', 'Field7', 'Field8', 'Field9', 'GeographicField10A', 'GeographicField10B', 'GeographicField11A', 'GeographicField11B', 'GeographicField12A', 'GeographicField12B', 'GeographicField13A', 'GeographicField13B', 'GeographicField14A', 'GeographicField14B', 'GeographicField15A', 'GeographicField15B', 'GeographicField16A', 'GeographicField16B', 'GeographicField17A', 'GeographicField17B', 'GeographicField18A', 'GeographicField18B', 'GeographicField19A', 'GeographicField19B', 'GeographicField1A', 'GeographicField1B', 'GeographicField20A', 'GeographicField20B', 'GeographicField21A', 'GeographicField21B', 'GeographicField22A', 'GeographicField22B', 'GeographicField23A', 'GeographicField23B', 'GeographicField24A',

**Pre-progressing**

In [11]:
# Separate Target column from Dataset
X_train = trainData.iloc[:, :-1]
y_train = trainData.iloc[:, -1] 

# Dropping "GeographicField64' from testset
X_test = testData.iloc[:, :-1]

**SMOTE**

In [12]:
#Smote ratio 0.8
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.8)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train_sm))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 42190})


**Modelling**

**1. Classifier eveluation**

model 1. Decision Tree

In [ ]:
dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train_sm, y_train_sm)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [ ]:
y_pred_dt=dt.predict_proba(X_test)[:,1]

In [ ]:
dt_clf_sm = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_dt
})


dt_clf_sm.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_dt_sm.csv', index=False) 


model 2. Random Forest

In [ ]:
rfc = RandomForestClassifier(random_state=1)  
rfc.fit(X_train_sm, y_train_sm)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
y_pred_rfc = rfc.predict_proba(X_test)[:,1]

In [ ]:
rfc_clf_sm = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_rfc
})

rfc_clf_sm.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_rfc_sm.csv', index=False) 

model 03. KNN

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_sm, y_train_sm)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_pred_knn = knn.predict_proba(X_test)[:,1]

In [ ]:
knn_clf_sm = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_knn
})

knn_clf_sm.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_knn_sm.csv', index=False) 

model 4. support vector machines

In [ ]:
svc = NuSVC()
svc.fit(X_train_sm, y_train_sm)

#time_consuming model : error with running 2h 29m 40s 

In [ ]:
svc = LinearSVC()
svc.fit(X_train_sm, y_train_sm)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
#LinearSVC do not support predict_proba

y_pred_svc = svc.predict(X_test)


In [ ]:
svc_clf_sm = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_svc
})

svc_clf_sm.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_svc_sm.csv', index=False) 

model 5. Multilayer perceptron 

In [ ]:
MLP = MLPClassifier(random_state=1, early_stopping=True)
MLP.fit(X_train_sm, y_train_sm)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
y_pred_mlp = MLP.predict_proba(X_test)[:,1]

In [ ]:
mlp_clf_sm = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_mlp
})

mlp_clf_sm.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_mlp_sm.csv', index=False) 

model 6. XGBoosting

In [ ]:
XGB = XGBClassifier (random_state=1)
XGB.fit(X_train_sm, y_train_sm)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred_xgb = XGB.predict_proba(X_test.values)[:,1]

In [ ]:
xgb_clf_sm = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_xgb
})

xgb_clf_sm.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_xgb_sm.csv', index=False) 

model 7. LogisticRegression

In [ ]:
LR_clf = LogisticRegression(random_state=1)
LR_clf.fit(X_train_sm, y_train_sm)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred_LR = LR_clf.predict_proba(X_test)[:,1]

In [ ]:
LR_clf_sm = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_LR
})

LR_clf_sm.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_LR_sm.csv', index=False) 

**2.Stacking**

In [14]:
#stacking with Top 5 indivisual model 

print("___________________________________________________________________________________________\nEnsemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression\n")

models = [ DecisionTreeClassifier(), RandomForestClassifier(), MLPClassifier(early_stopping=True), XGBClassifier(), LogisticRegression() ]
      
S_Train, S_Test = stacking(models,   
                           X_train_sm, y_train_sm, X_test.values,                
                           regression=False, 
                           mode='oof_pred_bag', 
                           needs_proba=True,
                           save_dir=None, 
                           metric=log_loss, 
                           n_folds=4, 
                           stratified=True,
                           shuffle=True,  
                           random_state=0,    
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [2.88744869]
    fold  1:  [2.90200237]
    fold  2:  [2.80740349]
    fold  3:  [2.84233231]
    ----
    MEAN:     [2.85979671] + [0.03740322]
    FULL:     [2.85979671]

model  1:     [RandomForestClassifier]
    fold  0:  [0.17527825]
    fold  1:  [0.17721537]
    fold  2:  [0.17498666]
    fold  3:  [0.17303009]
    ----
    MEAN:     [0.17512759] + [0.00148332]
    FULL:     [0.17512759]

model  2:     [MLPClassifier]
    fold  0:  [1.82676233]
    fold  1:  [1.56312580]
    fold  2:  [2.12085526]
    fold  3:  [2.18134509]
    ----
    MEAN:     [1.92302212] + [0.24731560]
    FULL:     [1.92302212]

model  3:     

In [9]:
#save stacking with Top 5 indivisual model 
model = XGBClassifier(random_state=1)
    
model = model.fit(S_Train, y_train_sm)
y_pred_meta_xgb = model.predict_proba(S_Test)[:,1]

meta_xgb = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_meta_xgb
})

meta_xgb.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_meta_xgb_with_5.csv', index=False) 

In [10]:
#stacking with only Top 2 indivisual model 

print("___________________________________________________________________________________________\nEnsemble Methods Predictions using RandomForest and XGBoosting\n")

models = [ RandomForestClassifier(), XGBClassifier()]
      
S_Train, S_Test = stacking(models,   
                           X_train_sm, y_train_sm, X_test.values,                
                           regression=False, 
                           mode='oof_pred_bag', 
                           needs_proba=True,
                           save_dir=None, 
                           metric=log_loss, 
                           n_folds=4, 
                           stratified=True,
                           shuffle=True,  
                           random_state=0,    
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using RandomForest and XGBoosting

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.17569284]
    fold  1:  [0.17556649]
    fold  2:  [0.17631519]
    fold  3:  [0.17422104]
    ----
    MEAN:     [0.17544889] + [0.00076345]
    FULL:     [0.17544889]

model  1:     [XGBClassifier]
    fold  0:  [0.14263676]
    fold  1:  [0.14313031]
    fold  2:  [0.14376564]
    fold  3:  [0.14221244]
    ----
    MEAN:     [0.14293629] + [0.00057861]
    FULL:     [0.14293629]



In [13]:
#save stacking with only Top 2 indivisual model 
model = XGBClassifier(random_state=1)
    
model = model.fit(S_Train, y_train_sm)
y_pred_meta_xgb = model.predict_proba(S_Test)[:,1]

meta_xgb = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_meta_xgb
})

meta_xgb.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_meta_xgb_with_2.csv', index=False) 

**3. Meta model building**

Meta model decision

In [ ]:
#meta_model_test = xgboost
model = XGBClassifier(random_state=1)
    
model = model.fit(S_Train, y_train_sm)
y_pred_meta_xgb = model.predict_proba(S_Test)[:,1]

meta_xgb = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_meta_xgb
})

meta_xgb.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_meta_xgb.csv', index=False) 


In [ ]:
#meta_model_test = Logistic Regression
model = LogisticRegression(random_state=1)

model = model.fit(S_Train, y_train_sm)
y_pred_meta_LR = model.predict_proba(S_Test)[:,1]

meta_LR = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_meta_LR
})

meta_LR.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_meta_LR.csv', index=False) 

SMOTE Experience with meta model


In [15]:
#Smote ratio 0.5
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train_sm))


#stacking model with Smote ratio 0.5
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression\n")

models = [ DecisionTreeClassifier(), RandomForestClassifier(), MLPClassifier(early_stopping=True), XGBClassifier(), LogisticRegression() ]
      
S_Train, S_Test = stacking(models,   
                           X_train_sm, y_train_sm, X_test.values,                
                           regression=False,    
                           mode='oof_pred_bag',        
                           needs_proba=True,         
                           save_dir=None,             
                           metric=log_loss,     
                           n_folds=4,                  
                           stratified=True,            
                           shuffle=True,              
                           random_state=0,       
                           verbose=2)

#prediction model
model = XGBClassifier(random_state=1)
    
model = model.fit(S_Train, y_train_sm)
y_pred_sm_05 = model.predict_proba(S_Test)[:,1]

xgb_meta_sm_05 = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_sm_05
})

xgb_meta_sm_05.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_xgb_meta_sm_5.csv', index=False)

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})
___________________________________________________________________________________________
Ensemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [3.36358817]
    fold  1:  [3.48059773]
    fold  2:  [3.48234414]
    fold  3:  [3.41440675]
    ----
    MEAN:     [3.43523420] + [0.04960880]
    FULL:     [3.43523446]

model  1:     [RandomForestClassifier]
    fold  0:  [0.20476548]
    fold  1:  [0.20508215]
    fold  2:  [0.21216397]
    fold  3:  [0.20399148]
    ----
    MEAN:     [0.20650077] + [0.00329364]
    FULL:     [0.20650080]

model  2:     [MLPClassifier]
    fold  0:  [2.80

In [16]:
#Smote ratio 0.9
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.9)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train_sm))

#stacking model with Smote ratio 0.9
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression\n")

models = [ DecisionTreeClassifier(), RandomForestClassifier(), MLPClassifier(early_stopping=True), XGBClassifier(), LogisticRegression() ]
      
S_Train, S_Test = stacking(models,   
                           X_train_sm, y_train_sm, X_test.values,                
                           regression=False,      
                           mode='oof_pred_bag',        
                           needs_proba=True,         
                           save_dir=None,             
                           metric=log_loss,     
                           n_folds=4,                  
                           stratified=True,            
                           shuffle=True,              
                           random_state=0,           
                           verbose=2)

#prediction model 

model = XGBClassifier(random_state=1)
    
model = model.fit(S_Train, y_train_sm)
y_pred_sm_09 = model.predict_proba(S_Test)[:,1]

xgb_meta_sm_09 = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_sm_09
})

xgb_meta_sm_09.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_xgb_meta_sm_9.csv', index=False) 

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 47464})
___________________________________________________________________________________________
Ensemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [2.70370606]
    fold  1:  [2.70784228]
    fold  2:  [2.81549626]
    fold  3:  [2.68864726]
    ----
    MEAN:     [2.72892297] + [0.05049098]
    FULL:     [2.72892250]

model  1:     [RandomForestClassifier]
    fold  0:  [0.16538507]
    fold  1:  [0.16329086]
    fold  2:  [0.16661374]
    fold  3:  [0.16660470]
    ----
    MEAN:     [0.16547359] + [0.00135568]
    FULL:     [0.16547357]

model  2:     [MLPClassifier]
    fold  0:  [1.86

In [18]:
#Smote ratio 0.4
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.4)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train_sm))

#stacking model with Smote ratio 0.4
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression\n")

models = [ DecisionTreeClassifier(), RandomForestClassifier(), MLPClassifier(early_stopping=True), XGBClassifier(), LogisticRegression() ]
      
S_Train, S_Test = stacking(models,   
                           X_train_sm, y_train_sm, X_test.values,                
                           regression=False,     
                           mode='oof_pred_bag',        
                           needs_proba=True,         
                           save_dir=None,             
                           metric=log_loss,     
                           n_folds=4,                  
                           stratified=True,            
                           shuffle=True,              
                           random_state=0,           
                           verbose=2)

model = XGBClassifier(random_state=1)
    
model = model.fit(S_Train, y_train_sm)
y_pred_sm_04 = model.predict_proba(S_Test)[:,1]

xgb_meta_sm_04 = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_sm_04
})

xgb_meta_sm_04.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_xgb_meta_sm_4.csv', index=False)

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 21095})
___________________________________________________________________________________________
Ensemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [3.58317118]
    fold  1:  [3.58897893]
    fold  2:  [3.61143344]
    fold  3:  [3.57400926]
    ----
    MEAN:     [3.58939820] + [0.01379604]
    FULL:     [3.58939812]

model  1:     [RandomForestClassifier]
    fold  0:  [0.21737091]
    fold  1:  [0.22099093]
    fold  2:  [0.21718522]
    fold  3:  [0.22241889]
    ----
    MEAN:     [0.21949149] + [0.00227122]
    FULL:     [0.21949146]

model  2:     [MLPClassifier]
    fold  0:  [2.17

In [19]:
#Smote ratio 0.3
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.3)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train_sm))

#stacking model with Smote ratio 0.3
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression\n")

models = [ DecisionTreeClassifier(), RandomForestClassifier(), MLPClassifier(early_stopping=True), XGBClassifier(), LogisticRegression() ]
      
S_Train, S_Test = stacking(models,   
                           X_train_sm, y_train_sm, X_test.values,                
                           regression=False,     
                           mode='oof_pred_bag',        
                           needs_proba=True,         
                           save_dir=None,             
                           metric=log_loss,     
                           n_folds=4,                  
                           stratified=True,            
                           shuffle=True,              
                           random_state=0,         
                           verbose=2)

#prediction model
model = XGBClassifier(random_state=1)
    
model = model.fit(S_Train, y_train_sm)
y_pred_sm_03 = model.predict_proba(S_Test)[:,1]

xgb_meta_sm_03 = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_sm_03
})

xgb_meta_sm_03.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_xgb_meta_sm_3.csv', index=False) 

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 15821})
___________________________________________________________________________________________
Ensemble Methods Predictions using DecisionTree, RandomForest, MLP, XGBoosting and LogisticRegression

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [3.82264054]
    fold  1:  [3.85286701]
    fold  2:  [3.89719916]
    fold  3:  [3.92160913]
    ----
    MEAN:     [3.87357896] + [0.03836834]
    FULL:     [3.87357826]

model  1:     [RandomForestClassifier]
    fold  0:  [0.23360050]
    fold  1:  [0.23287578]
    fold  2:  [0.23552168]
    fold  3:  [0.22533671]
    ----
    MEAN:     [0.23183367] + [0.00387361]
    FULL:     [0.23183376]

model  2:     [MLPClassifier]
    fold  0:  [2.27

Meta model hyper parameter tunning

In [20]:
#Randomize Search_1

XGB = XGBClassifier(random_state=1)

parameters = {'learning_rate':[0.3,0.01,10], 'normalize_type':['forest', 'tree']}

model_random = RandomizedSearchCV(XGB,parameters,n_iter=15)
model_random.fit(S_Train, y_train_sm)
grid_parm=model_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning

model = XGBClassifier(**grid_parm)
model.fit(S_Train, y_train_sm)
y_model_pred_random = model.predict_proba(S_Test)[:,1]

model_predict_random = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_model_pred_random
})

model_predict_random.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_meta_xgb_random_1.csv', index=False) 


{'normalize_type': 'forest', 'learning_rate': 0.3}


In [23]:
#Randomize Search_2

XGB = XGBClassifier(random_state=1)
parameters = {'scale_pos_weight':[0.1,0.01,0.001], 'gamma':[0,1,100],'grow_policy':['depthwise','lossguide']}

model_random = RandomizedSearchCV(XGB,parameters,n_iter=15)
model_random.fit(S_Train, y_train_sm)
grid_parm=model_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning 

model = XGBClassifier(**grid_parm)
model.fit(S_Train, y_train_sm)
y_model_pred_random = model.predict_proba(S_Test)[:,1]

model_predict_random = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_model_pred_random
})

model_predict_random.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_meta_xgb_random_2.csv', index=False) 

{'scale_pos_weight': 0.1, 'grow_policy': 'lossguide', 'gamma': 0}


**Best Stacked Model**

In [ ]:
# Best model 
sm = SMOTE(sampling_strategy='float', ratio=0.3)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

models = [ DecisionTreeClassifier(), RandomForestClassifier(), 
          MLPClassifier(early_stopping=True), XGBClassifier(), LogisticRegression() ]

S_Train, S_Test = stacking(models,   
                           X_train_sm, y_train_sm, X_test.values,                
                           regression=False, 
                           mode='oof_pred_bag',   
                           needs_proba=True,         
                           save_dir=None,             
                           metric=log_loss,    
                           n_folds=4,                  
                           stratified=True,            
                           shuffle=True,              
                           random_state=0,          
                           verbose=2)

model = XGBClassifier(random_state=1,scale_pos_weight=0.1, grow_policy ='lossguide') 
model = model.fit(S_Train, y_train_sm)
y_pred_final = model.predict_proba(S_Test)[:,1]
xgb_meta_final = pd.DataFrame ({
    'QuoteNumber':testData['QuoteNumber'],
    'QuoteConversion_Flag': y_pred_final
})
xgb_meta_final.to_csv('/content/drive/MyDrive/CIS508/Assignment_3/ submit/trial_meta_final.csv', index=False) 

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [3.55323346]
    fold  1:  [3.64885762]
    fold  2:  [3.61330465]
    fold  3:  [3.54032750]
    ----
    MEAN:     [3.58893081] + [0.04422072]
    FULL:     [3.58893032]

model  1:     [RandomForestClassifier]
    fold  0:  [0.22290059]
    fold  1:  [0.22310191]
    fold  2:  [0.21689046]
    fold  3:  [0.22476049]
    ----
    MEAN:     [0.22191336] + [0.00298843]
    FULL:     [0.22191338]

model  2:     [MLPClassifier]
    fold  0:  [3.47305318]
    fold  1:  [2.35377292]
    fold  2:  [1.97525675]
    fold  3:  [2.40500833]
    ----
    MEAN:     [2.55177280] + [0.55719657]
    FULL:     [2.55178527]

model  3:     [XGBClassifier]
    fold  0:  [0.17807064]
    fold  1:  [0.17852418]
    fold  2:  [0.17525140]
    fold  3:  [0.17698653]
    ----
    MEAN:     [0.17720819] + [0.00126034]
    FULL:     [0.177